 PPからlocationシーケンスの生成

In [7]:
import pandas as pd 
import numpy as np 
import os 
import sys
import geopandas as gpd
from shapely.geometry import Point

time, mesh, actの個人ごとのトータルシーケンスを作成（この後に1週間ごとにデータ分割）

In [8]:
# trip dataの出発時刻に併せて loc_dataからlon, latを取り出す
base_path0 = '/Users/matsunagatakahiro/Desktop/jrres/PPcameraTG/gpslog'#/04_202212old'
input_path = '/Users/matsunagatakahiro/Desktop/res2025/ActFormer/RoutesFormer/actgpt0512/input'
dataset_list = ['04_202212old', '04_202301new', '05_202311', 'toyosu_2019/201907-202002', '99_202110/logs']
nameid_list = ['01', '02', '03', 'toyosu', 'shibu21']
gdf_mesh1 = gpd.read_file(os.path.join(base_path0, '3rdmesh5339/MESH05339.shp'))
gdf_mesh2 = gpd.read_file(os.path.join(base_path0, '3rdmesh5340/MESH05340.shp'))
gdf_mesh3 = gpd.read_file(os.path.join(base_path0, '3rdmesh5440/MESH05440.shp'))
gdf_mesh4 = gpd.read_file(os.path.join(base_path0, '3rdmesh5239/MESH05239.shp'))
gdf_mesh5 = gpd.read_file(os.path.join(base_path0, '3rdmesh5439/MESH05439.shp'))

gdf_mesh = pd.concat([gdf_mesh1, gdf_mesh2, gdf_mesh3, gdf_mesh4, gdf_mesh5], ignore_index=True)
gdf_mesh = gdf_mesh.to_crs("EPSG:4326") # WGS84に変換
gdf_mesh['3rdmesh'] = gdf_mesh['KEY_CODE'].astype(str).str[0:6] # 3次メッシュコード

act_dict_shibuya2223 = {
    110: 1, # H
    200: 2, # S
    210: 2, # S
    220: 1, # hotel->home
    300: 3, # W 
    310: 4, # W2
    400: 3, # W
    998: 5, # Other
    999: 5 # , # Other
}

act_dict_shibuya21toyosu = {
    300: 1, # H
    201: 1, # H
    400: 2, # S
    401: 2, 
    402: 2, 
    404: 2, 
    405: 2, 
    406: 2, 
    100: 3, # W
    200: 4, # W2
    403: 5, # Ohter
    407: 5, # Ohter
    500: 5, # Ohter
    501: 5, # Ohter
    999: 5 #, # Ohter`
}

def remove_consecutive_duplicates(act_row, time_row, mesh_row):
    # print('remove_consecutive_duplicates', act_row, time_row, mesh_row)
    new_act = [act_row[0]]
    new_time = [time_row[0]]
    new_mesh = [mesh_row[0]]
    for i in range(1, len(act_row)):
        if not (act_row[i] == act_row[i-1] and mesh_row[i] == mesh_row[i-1]):
            new_act.append(act_row[i])
            new_time.append(time_row[i])
            new_mesh.append(mesh_row[i])
    return pd.Series(new_act), pd.Series(new_time), pd.Series(new_mesh)

def try_convert_int(x):
    if x in ['<b>', '<e>']:
        return x
    try:
        return int(float(x))  # 300.0 → 300 などにも対応
    except:
        return x  # 変換できなければそのまま


In [ ]:

for datasetid, dataset in enumerate(dataset_list):
    base_path = os.path.join(base_path0, dataset)
    folderlist = os.listdir(base_path)
    user_log_dict = {}
    user_count_dict = {} # 何日登場したかのカウント
    user_mesh_traj = {}
    user_time_traj = {}
    user_act_traj = {}
    nokitaku_count = 0
    all_count = 0
    folderlist = sorted(folderlist)

    if dataset == 'toyosu_2019/201907-202002' or dataset == '99_202110/logs': 
        act_dict = act_dict_shibuya21toyosu
    elif dataset == '04_202212old' or dataset == '04_202301new' or dataset == '05_202311':
        act_dict = act_dict_shibuya2223

    for folder in folderlist: # 個々の日
        if not os.path.isdir(os.path.join(base_path, folder)):
            continue

        filelist = os.listdir(os.path.join(base_path, folder))
        
        trip_path = os.path.join(base_path, folder, 't_trip.csv')
        loc_path = os.path.join(base_path, folder, 't_loc_data.csv')
        # ファイルがなければスキップ
        if not os.path.exists(trip_path) or not os.path.exists(loc_path):
            print(f"スキップ: {folder}（ファイルが存在しない）")
            continue

        df_trip = pd.read_csv(trip_path, encoding='shift-jis')
        df_loc = pd.read_csv(loc_path, encoding='shift-jis')
        df_loc['記録日時'] = pd.to_datetime(df_loc['記録日時'].str.split('.').str[0], errors='coerce') # 秒の小数点以下を切り捨て
        df_trip['出発時刻'] = pd.to_datetime(df_trip['出発時刻'])
        df_trip['到着時刻'] = pd.to_datetime(df_trip['到着時刻'])
        grouped = df_trip.groupby('ユーザーID')

        #### メッシュコードを取得 ####        
        gdf_points = gpd.GeoDataFrame(df_loc, geometry=gpd.points_from_xy(df_loc['経度'], df_loc['緯度']), crs="EPSG:4326")
        gdf_joined = gpd.sjoin(gdf_points, gdf_mesh, predicate='within')
        df_loc2 = df_loc.copy()
        df_loc2['mesh'] = gdf_joined['3rdmesh']
        
        for user_id, group in grouped: # 個人ごと
            userid = group['ユーザーID'].values[0]
            group = group.sort_values(by='出発時刻')
            acts = list(group['目的コード（active）']) # 系列
            times = list(group['出発時刻']) # 系列
            
            #### 異常値排除 #### # タイミングによって処理が異なる
            all_count += 1
            if dataset == 'toyosu_2019/201907-202002' or dataset == '99_202110/logs': 
                if acts[-1] != 300 and acts[-1] != 201:
                    if acts[-1] == 500 or acts[-1] == 501 or acts[-1] == 999:
                        acts[-1] = 300
                        # continue
                    # print('kitakushitenai!!!')
                if acts[-1] != 300 and acts[-1] != 201:
                    nokitaku_count += 1
                    continue
                # 最初から帰宅の人
                if acts[0] == 300 or acts[0] == 201:
                    nokitaku_count += 1
                    continue

            elif dataset == '04_202212old' or dataset == '04_202301new' or dataset == '05_202311':
                if acts[-1] != 110:
                    if acts[-1] == 998 or acts[-1] == 999:
                        acts[-1] = 110
                if acts[-1] != 110:
                    nokitaku_count += 1
                    continue
                if acts[0] == 110:
                    nokitaku_count += 1
                    continue

            locmesh = []
            for i, time in enumerate(times):
                time = pd.to_datetime(time, errors='coerce')  # ← ここで変換
                user_loc_df = df_loc2[df_loc2['ユーザーID'] == userid].copy()

                if user_loc_df.empty or pd.isna(time):
                    continue
                # 時刻差を計算
                user_loc_df['time_diff'] = (user_loc_df['記録日時'] - time).abs()

                # 最小差分の行を取得（NaTは自動除外される）
                nearest_row = user_loc_df.loc[user_loc_df['time_diff'].idxmin()]
                mesh = nearest_row['mesh']
                #print(f"User ID: {userid}, Time: {time}, mesh: {mesh}")
                locmesh.append(mesh)
            
            # timeは1時間おきにする
            times = list(group['出発時刻'].dt.floor('h'))
            locmesh.append('<e>')
            locmesh.insert(0, '<b>') #OK
            acts.insert(0, '<b>')
            acts.append('<e>')
            times.insert(0, '<b>')
            times.append('<e>')

            if user_id not in list(user_log_dict.keys()):
                user_log_dict[user_id] = len(group)
                user_count_dict[user_id] = 1
                user_mesh_traj[user_id] = locmesh
                user_time_traj[user_id] = times
                user_act_traj[user_id] = acts

            else:
                user_log_dict[user_id] += len(group)
                user_count_dict[user_id] += 1
                user_mesh_traj[user_id] += locmesh
                user_time_traj[user_id] += times
                user_act_traj[user_id] += acts

    # insertしたusrer_idがindexにならないように処理
    # df_mesh_traj = pd.DataFrame.from_dict(user_mesh_traj, orient='index')
    df_mesh_traj = pd.DataFrame.from_dict(user_mesh_traj, orient='index').reset_index(names='user_id')
    # df_time_traj = pd.DataFrame.from_dict(user_time_traj, orient='index')
    df_time_traj = pd.DataFrame.from_dict(user_time_traj, orient='index').reset_index(names='user_id')
    # df_act_traj = pd.DataFrame.from_dict(user_act_traj, orient='index')
    df_act_traj = pd.DataFrame.from_dict(user_act_traj, orient='index').reset_index(names='user_id')

    # print('1', df_mesh_traj.head(5))
    # print('2test', df_mesh_traj.iloc[0].tolist())

    # df_mesh_traj.insert(0, 'user_id', df_mesh_traj.index)
    # df_time_traj.insert(0, 'user_id', df_time_traj.index)
    # df_act_traj.insert(0, 'user_id', df_act_traj.index)

    #### 活動ラベルを集約 ####
    df_act_traj = df_act_traj.applymap(try_convert_int)
    df_act_traj = df_act_traj.applymap(lambda x: act_dict.get(x, x))

    # print('1', df_mesh_traj.head(5))
    # print('2test', df_mesh_traj.iloc[0].tolist())
    # print()
    
    #### 活動場所と種類が連続している場合は削除 ####
    # 最大列数（不足分を埋める用）
    max_len = max(df_act_traj.shape[1], df_time_traj.shape[1], df_mesh_traj.shape[1])

    # NaNを埋める（列数を揃える）
    # df_act_traj = df_act_traj.reindex(columns=range(max_len))
    # df_time_traj = df_time_traj.reindex(columns=range(max_len))
    # df_mesh_traj = df_mesh_traj.reindex(columns=range(max_len))
    
    # 'user_id' 以外の列だけを対象に reindex してから結合
    id_col = df_act_traj[['user_id']]
    act_cols = df_act_traj.drop(columns='user_id')
    act_cols = act_cols.reindex(columns=range(max_len))
    df_act_traj = pd.concat([id_col, act_cols], axis=1)

    # 同様に
    id_col = df_time_traj[['user_id']]
    time_cols = df_time_traj.drop(columns='user_id')
    time_cols = time_cols.reindex(columns=range(max_len))
    df_time_traj = pd.concat([id_col, time_cols], axis=1)

    id_col = df_mesh_traj[['user_id']]
    mesh_cols = df_mesh_traj.drop(columns='user_id')
    mesh_cols = mesh_cols.reindex(columns=range(max_len))
    df_mesh_traj = pd.concat([id_col, mesh_cols], axis=1)
    
    # print('2', df_mesh_traj.head(5))
    act_cleaned = []
    time_cleaned = []
    mesh_cleaned = []

    # print('2222df_act_traj', df_act_traj.iloc[0].tolist())
    # print('222', df_time_traj.columns)
    # sys.exit()

    for i in range(len(df_act_traj)):
        a_row = df_act_traj.iloc[i].tolist()
        t_row = df_time_traj.iloc[i].tolist()
        m_row = df_mesh_traj.iloc[i].tolist()
        a_new, t_new, m_new = remove_consecutive_duplicates(a_row, t_row, m_row)
        act_cleaned.append(a_new)
        time_cleaned.append(t_new)
        mesh_cleaned.append(m_new)

    # データフレーム化して保存
    df_act_cleaned = pd.DataFrame(act_cleaned)
    df_time_cleaned = pd.DataFrame(time_cleaned)
    df_mesh_cleaned = pd.DataFrame(mesh_cleaned)
    # print('3', df_mesh_cleaned.head(5))
    # sys.exit()

    df_mesh_cleaned.iloc[1:, :].to_csv(os.path.join(input_path, f'meshtraj{nameid_list[datasetid]}.csv'), index=False) 
    df_time_cleaned.iloc[1:, :].to_csv(os.path.join(input_path, f'timetraj{nameid_list[datasetid]}.csv'), index=False)
    df_act_cleaned.iloc[1:, :].to_csv(os.path.join(input_path, f'acttraj{nameid_list[datasetid]}.csv'), index=False)
    



/var/folders/_k/qgnv6b3961q5nz5l1v78bw080000gn/T/ipykernel_36207/1373987240.py:197: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_act_traj = df_act_traj.applymap(try_convert_int)
/var/folders/_k/qgnv6b3961q5nz5l1v78bw080000gn/T/ipykernel_36207/1373987240.py:198: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_act_traj = df_act_traj.applymap(lambda x: act_dict.get(x, x))
/var/folders/_k/qgnv6b3961q5nz5l1v78bw080000gn/T/ipykernel_36207/1373987240.py:197: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_act_traj = df_act_traj.applymap(try_convert_int)
/var/folders/_k/qgnv6b3961q5nz5l1v78bw080000gn/T/ipykernel_36207/1373987240.py:198: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_act_traj = df_act_traj.applymap(lambda x: act_dict.get(x, x))
/var/folders/_k/qgnv6b3961q5nz5l1v78bw080000gn/T/ipykernel_36207/1373987240.py:197: FutureWarn

スキップ: 10004_20190915（ファイルが存在しない）
スキップ: 10004_20190916（ファイルが存在しない）
スキップ: 10004_20191020（ファイルが存在しない）
スキップ: 10004_20191229（ファイルが存在しない）
スキップ: 10004_20191230（ファイルが存在しない）
スキップ: 10004_20191231（ファイルが存在しない）
スキップ: 10004_20200101（ファイルが存在しない）
スキップ: 10004_20200102（ファイルが存在しない）
スキップ: 10004_20200103（ファイルが存在しない）
スキップ: 10004_20200104（ファイルが存在しない）
スキップ: 10004_20200105（ファイルが存在しない）
スキップ: 10004_20200111（ファイルが存在しない）
スキップ: 10004_20200112（ファイルが存在しない）
スキップ: 10004_20200113（ファイルが存在しない）
スキップ: 10004_20200126（ファイルが存在しない）
スキップ: 10004_20200127（ファイルが存在しない）
スキップ: 10004_20200201（ファイルが存在しない）
スキップ: 10004_20200202（ファイルが存在しない）
スキップ: 10004_20200208（ファイルが存在しない）
スキップ: 10004_20200209（ファイルが存在しない）
スキップ: 10004_20200210（ファイルが存在しない）
スキップ: 10004_20200211（ファイルが存在しない）


/var/folders/_k/qgnv6b3961q5nz5l1v78bw080000gn/T/ipykernel_36207/1373987240.py:197: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_act_traj = df_act_traj.applymap(try_convert_int)
/var/folders/_k/qgnv6b3961q5nz5l1v78bw080000gn/T/ipykernel_36207/1373987240.py:198: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_act_traj = df_act_traj.applymap(lambda x: act_dict.get(x, x))
/var/folders/_k/qgnv6b3961q5nz5l1v78bw080000gn/T/ipykernel_36207/1373987240.py:197: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_act_traj = df_act_traj.applymap(try_convert_int)
/var/folders/_k/qgnv6b3961q5nz5l1v78bw080000gn/T/ipykernel_36207/1373987240.py:198: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_act_traj = df_act_traj.applymap(lambda x: act_dict.get(x, x))


1週間ごとに分割

In [ ]:
# 基礎関数の用意
from datetime import datetime, timedelta
import csv

def split_line_to_segments(line):
    """
    各行のトークン列を '<b>'／'<e>' で分割し、セグメントリストを返す
    """
    segments = []
    current = []
    for token in line:
        if token == '<b>':
            current = ['<b>']
        elif token == '<e>':
            if current:
                current.append('<e>')
                segments.append(current)
        else:
            current.append(token)
    return segments 


def flatten_chunks(chunks):
    """
    チャンク化されたセグメントを '<b>' と '<e>' 付きのフラットな行リストに変換
    """
    rows = []
    for chunk in chunks:
        row = ['<b>']
        for seg in chunk:
            row.extend(seg)
        row.append('<e>')
        rows.append(row)
    return rows


def date_of_time_seg(seg):
    """時間セグメントから日付を抽出"""
    return datetime.strptime(seg[1], "%Y-%m-%d %H:%M:%S").date() # 最初のセグメントだけidが入ってるので3番目の項から始まる


def group_into_weeks(segments): #, date_extractor):
    """
    7日間ごとにセグメントをグループ化
    """
    if not segments:
        return []
    chunks = []
    current = []
    start_date = datetime.strptime(segments[0][2], "%Y-%m-%d %H:%M:%S").date() # 最初のセグメントだけidが入ってるので3番目の項から始まる ok

    for i, seg in enumerate(segments):
        d = date_of_time_seg(seg)
        if d < start_date + timedelta(days=7):
            current.append(seg)
        else:
            chunks.append(current)
            current = [seg]
            start_date = d
        
    if current:
        chunks.append(current)

    return chunks


def fill_missing_days(weekly_time_segs, weekly_act_segs, weekly_mesh_segs):
    """
    欠落日にはプレースホルダーを挿入（time: 日付トークン / act: '955'）
    """
    # date -> (time_seg, act_seg)
    date_map = {}
    for t_seg, a_seg, m_seg in zip(weekly_time_segs, weekly_act_segs, weekly_mesh_segs):
        date_map[date_of_time_seg(t_seg)] = (t_seg, a_seg, m_seg)

    start = date_of_time_seg(weekly_time_segs[0])
    filled_time, filled_act, filled_mesh = [], [], []
    for i in range(7):
        d = start + timedelta(days=i)
        if d in date_map:
            t_seg, a_seg, m_seg = date_map[d]
        else:
            t_seg = ['<b>', d.strftime("%Y-%m-%d"), '<e>']
            a_seg = ['<b>', "955", '<e>']
            m_seg = ['<b>', "999999", '<e>']

        filled_time.append(t_seg)
        filled_act.append(a_seg)
        filled_mesh.append(m_seg)

    return filled_time, filled_act, filled_mesh


def write_chunks_to_csv(chunks, output_path):
    with open(output_path, 'w', newline='') as f:
        writer = csv.writer(f)
        for segs in chunks:
            row = ['<b>'] + [token for seg in segs for token in seg] + ['<e>']
            writer.writerow(row[1:])

In [40]:
# 各データセットに対して実行
for datasetid, dataset in enumerate(dataset_list):
    base_path = os.path.join(base_path0, dataset)
    folderlist = os.listdir(base_path)
    act_file = (os.path.join(input_path, f'acttraj{nameid_list[datasetid]}.csv'))
    time_file = (os.path.join(input_path, f'timetraj{nameid_list[datasetid]}.csv'))
    mesh_file = (os.path.join(input_path, f'meshtraj{nameid_list[datasetid]}.csv'))

    out_time_path = os.path.join(input_path, f'time_weekly{nameid_list[datasetid]}.csv')
    out_act_path = os.path.join(input_path, f'act_weekly{nameid_list[datasetid]}.csv')
    out_mesh_path = os.path.join(input_path, f'mesh_weekly{nameid_list[datasetid]}.csv')
    
    # csvを読み込む：先頭はuseridなので飛ばす
    with open(time_file, newline='') as f:
        time_rows = list(csv.reader(f))[1:]
    with open(act_file, newline='') as f:
        act_rows = list(csv.reader(f))[1:]
    with open(mesh_file, newline='') as f:
        mesh_rows = list(csv.reader(f))[1:]

    all_time_chunks, all_act_chunks, all_mesh_chunks = [], [], []
    count = 0 
    for t_row, a_row, m_row in zip(time_rows, act_rows, mesh_rows): # 個人ごと
        t_segs = split_line_to_segments(t_row) # idは消えてる
        a_segs = split_line_to_segments(a_row)
        m_segs = split_line_to_segments(m_row)

        ## これある？？
        userid = (t_row[0]) # fixed
        weekly_ts = group_into_weeks(t_segs) # 日毎のb-eのリスト
        idx_map = {tuple(seg): idx for idx, seg in enumerate(t_segs)}

        for week in weekly_ts:
            # 元データのactを対応づけ
            act_week = []
            mesh_week = []
            for seg in week:
                idx = idx_map.get(tuple(seg))
                act_week.append(a_segs[idx] if idx is not None else ["955"])
                mesh_week.append(m_segs[idx] if idx is not None else ["999999"])

            # 欠落日の補完
            filled_t, filled_a, filled_m = fill_missing_days(week, act_week, mesh_week)

            filled_t[0].insert(0, f'{userid}')
            filled_a[0].insert(0, f'{userid}')
            filled_m[0].insert(0, f'{userid}')

            all_time_chunks.append(filled_t)
            all_act_chunks.append(filled_a)
            all_mesh_chunks.append(filled_m)

    write_chunks_to_csv(all_time_chunks, out_time_path)
    write_chunks_to_csv(all_act_chunks, out_act_path)
    write_chunks_to_csv(all_mesh_chunks, out_mesh_path)

長さを揃えてから，5つのデータを統合する

In [41]:
act_all_rows = [] 
time_all_rows = [] 
mesh_all_rows = [] 

act_max_cols = 0
time_max_cols = 0
mesh_max_cols = 0

for datasetid, dataset in enumerate(dataset_list):
    base_path = os.path.join(base_path0, dataset)
    folderlist = os.listdir(base_path)

    time_path = os.path.join(input_path, f'time_weekly{nameid_list[datasetid]}.csv')
    act_path = os.path.join(input_path, f'act_weekly{nameid_list[datasetid]}.csv')
    mesh_path = os.path.join(input_path, f'mesh_weekly{nameid_list[datasetid]}.csv')

    # 読み込み + 空文字・None・"NaN"除去
    with open(act_path, newline='') as f:
        reader = csv.reader(f)
        act_cleaned_rows = [[cell for cell in row if cell not in [None, '', 'NaN']] for row in reader]
    
    with open(time_path, newline='') as f:
        reader = csv.reader(f)
        time_cleaned_rows = [[cell for cell in row if cell not in [None, '', 'NaN']] for row in reader]

    with open(mesh_path, newline='') as f:
        reader = csv.reader(f)
        mesh_cleaned_rows = [[cell for cell in row if cell not in [None, '', 'NaN']] for row in reader]
    
    # 最大列数を更新
    for row in act_cleaned_rows:
        act_max_cols = max(act_max_cols, len(row))
    
    for row in time_cleaned_rows:
        time_max_cols = max(time_max_cols, len(row))

    for row in mesh_cleaned_rows:
        mesh_max_cols = max(mesh_max_cols, len(row))
    
    act_all_rows.extend(act_cleaned_rows)
    time_all_rows.extend(time_cleaned_rows)
    mesh_all_rows.extend(mesh_cleaned_rows)

# 行の長さを最大列数にそろえ、<p>で埋める
act_padded_rows = [row + ['<p>'] * (act_max_cols - len(row)) for row in act_all_rows]
time_padded_rows = [row + ['<p>'] * (time_max_cols - len(row)) for row in time_all_rows]
mesh_padded_rows = [row + ['<p>'] * (mesh_max_cols - len(row)) for row in mesh_all_rows]

# 結合結果を保存
act_output_path = os.path.join(input_path, 'act_weekly_filled.csv')
time_output_path = os.path.join(input_path, 'time_weekly_filled.csv')
mesh_output_path = os.path.join(input_path, 'mesh_weekly_filled.csv')

with open(act_output_path, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(act_padded_rows)

with open(time_output_path, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(time_padded_rows)

with open(mesh_output_path, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(mesh_padded_rows)

print(f"🧱 総行数: {len(act_padded_rows)}, {len(time_padded_rows)}, {len(mesh_padded_rows)}")

🧱 総行数: 3992, 3992, 3992


In [42]:
def convert_datetime_to_hour(token):
    try:
        # 時刻が含まれる場合（フォーマット付き日付）
        dt = pd.to_datetime(token, errors='raise')
        if dt.hour == 0 and "00:00:00" not in token:
            return 25 # token  # たとえば "2022-12-21" のような場合
        return str(dt.hour)
    except Exception:
        # 変換できない場合はそのまま返す
        return token
## 
df_time = pd.read_csv(os.path.join(input_path, 'time_weekly_filled.csv'), header=None)
df_time = df_time.applymap(lambda x: convert_datetime_to_hour(x) if pd.notna(x) else x)
df_time.to_csv(os.path.join(input_path, 'time_weekly_filled_hour.csv'), index=False, header=False)
# converted_row = [convert_datetime_to_hour(tok) for tok in row]


/var/folders/_k/qgnv6b3961q5nz5l1v78bw080000gn/T/ipykernel_36207/1425381383.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_time = df_time.applymap(lambda x: convert_datetime_to_hour(x) if pd.notna(x) else x)


日付変更時点をSトークンに変換

In [6]:
df_act = pd.read_csv(os.path.join(input_path, 'act_weekly_filled.csv'), header=None)
df_time = pd.read_csv(os.path.join(input_path, 'time_weekly_filled_hour.csv'), header=None)
df_mesh = pd.read_csv(os.path.join(input_path, 'mesh_weekly_filled.csv'), header=None)

# 各行見ていって，<b>を消す＆<e>を<S>に置換
df_act = df_act.replace('<e>', '<s>')
df_time = df_time.replace('<e>', '<s>')
df_mesh = df_mesh.replace('<e>', '<s>')

# 特定の文字を削除して左詰め（文字列結合で再構成）
def remove_and_shift(row, char='<b>'):
    # 各セルから特定文字を削除し、行を再構成
    cleaned = [str(cell).replace(char, '') for cell in row]
    # 空でないものを左に寄せ、空文字を右に詰める
    non_empty = [c for c in cleaned if c]
    empty = [''] * (len(row) - len(non_empty))
    return pd.Series(non_empty + empty)

# 各行に適用
df_act_cleaned = df_act.apply(remove_and_shift, axis=1)
df_time_cleaned = df_time.apply(remove_and_shift, axis=1)
df_mesh_cleaned = df_mesh.apply(remove_and_shift, axis=1)

df_act_cleaned = df_act_cleaned.fillna('<p>')
df_time_cleaned = df_time_cleaned.fillna('<p>')
df_mesh_cleaned = df_mesh_cleaned.fillna('<p>')
# print(df_act_cleaned)

df_act_cleaned = df_act_cleaned.replace(['', 'NaN'], '<p>') #, inplace=True)
df_time_cleaned = df_time_cleaned.replace(['', 'NaN'], '<p>') #, inplace=True)
df_mesh_cleaned = df_mesh_cleaned.replace(['', 'NaN'], '<p>') #, inplace=True)

df_act = df_act_cleaned
df_time = df_time_cleaned
df_mesh = df_mesh_cleaned

#### ここから <s> を <s1>, <s2>, ... に変換 ####
def convert_d(row):
    # row ごとにカウント変えるので
    count = 0
    for i in range(len(row)):
        if row[i] == '<s>':
            count += 1
            if count <= 6: # countが7, 8の時は一番後ろのやつなので無視する
                row[i] = '<s' + str(count) + '>'

            else:
                pass

    return row

for i in range(len(df_time)):
    # 1行ずつ取り出す
    row = df_time.iloc[i]
    # 1行の中に<s>がいくつあるかをカウント
    row = convert_d(row)
    # 変換した行を元のデータフレームに戻す
    df_time.iloc[i] = row

for i in range(len(df_act)):
    # 1行ずつ取り出す
    row = df_act.iloc[i]
    # 1行の中に<s>がいくつあるかをカウント
    row = convert_d(row)
    # 変換した行を元のデータフレームに戻す
    df_act.iloc[i] = row

for i in range(len(df_mesh)):
    # 1行ずつ取り出す
    row = df_mesh.iloc[i]
    # 1行の中に<s>がいくつあるかをカウント
    row = convert_d(row)
    # 変換した行を元のデータフレームに戻す
    df_mesh.iloc[i] = row

df_time.to_csv(os.path.join(input_path, 'time_weekly_filled_structured.csv'), index=False)
df_act.to_csv(os.path.join(input_path, 'act_weekly_filled_structured.csv'), index=False)
df_mesh.to_csv(os.path.join(input_path, 'mesh_weekly_filled_structured.csv'), index=False)

ユニークなmeshの数をカウント

In [12]:
df_mesh = pd.read_csv(os.path.join(input_path, 'mesh_weekly_filled_structured.csv'))
print(df_mesh)


          0       1       2       3       4       5       6       7       8  \
0     23002  533946  533946  533946  533946  533946  533946    <s1>  533946   
1     23002  533946  533946  533946  533946  533935  533946  533946    <s1>   
2     23003  533945  533945    <s1>  999999    <s2>  999999    <s3>  533933   
3     23003  533934  533935    <s1>  999999    <s2>  999999    <s3>  999999   
4     23003  533933  533935    <s1>  999999    <s2>  999999    <s3>  999999   
...     ...     ...     ...     ...     ...     ...     ...     ...     ...   
3987  21402  533934  533914  533945    <s1>  999999    <s2>  999999    <s3>   
3988  21402  533946  533945    <s1>  999999    <s2>  999999    <s3>  999999   
3989  21415    <s1>  999999    <s2>  999999    <s3>  999999    <s4>  999999   
3990  21457  533945  533935  533945  533945    <s1>  999999    <s2>  999999   
3991  21550  533946  533946  533946  533946    <s1>  999999    <s2>  999999   

           9  ...   78   79   80   81   82   83   8

In [ ]:
df_mesh_val = df_mesh.iloc[:, 1:].values

nunique_mesh = np.unique(df_mesh_val)
print(nunique_mesh)
print('unique mesh num:', len(nunique_mesh))

['523910' '523930' '523936' '523947' '523950' '523956' '523960' '523965'
 '523966' '523967' '523971' '523972' '523973' '523974' '523975' '523976'
 '533901' '533902' '533903' '533904' '533905' '533907' '533911' '533912'
 '533913' '533914' '533915' '533916' '533917' '533922' '533923' '533924'
 '533925' '533926' '533930' '533931' '533932' '533933' '533934' '533935'
 '533936' '533937' '533941' '533942' '533943' '533944' '533945' '533946'
 '533947' '533950' '533951' '533952' '533953' '533954' '533955' '533956'
 '533957' '533961' '533962' '533963' '533964' '533965' '533966' '533967'
 '533970' '533973' '533974' '533975' '533976' '533977' '534001' '534003'
 '534010' '534011' '534012' '534020' '534021' '534022' '534023' '534030'
 '534031' '534032' '534040' '534041' '534042' '534045' '534050' '534051'
 '534052' '534053' '534060' '534063' '534065' '534071' '543902' '543903'
 '543904' '543905' '543906' '543912' '543913' '543914' '543915' '543922'
 '543923' '543924' '543925' '543930' '543931' '5439

In [15]:
gdf_mesh5 = gpd.read_file(os.path.join(base_path0, '3rdmesh5439/MESH05439.shp'))
print(gdf_mesh5.head(5))

   KEY_CODE MESH1_ID MESH2_ID MESH3_ID  OBJ_ID  \
0  54390000     5439       00       00       1   
1  54390001     5439       00       01       2   
2  54390002     5439       00       02       3   
3  54390003     5439       00       03       4   
4  54390004     5439       00       04       5   

                                            geometry  
0  POLYGON ((139.0125 36, 139 36, 139 36.00833, 1...  
1  POLYGON ((139.025 36, 139.0125 36, 139.0125 36...  
2  POLYGON ((139.0375 36, 139.025 36, 139.025 36....  
3  POLYGON ((139.05 36, 139.0375 36, 139.0375 36....  
4  POLYGON ((139.0625 36, 139.05 36, 139.05 36.00...  


In [16]:
df_act = pd.read_csv(os.path.join(input_path, 'act_weekly_filled_structured.csv'), header=None)
print(df_act.iloc[:, 1])

0       1.0
1       3.0
2       2.0
3       3.0
4       2.0
       ... 
3988    3.0
3989    2.0
3990    2.0
3991    3.0
3992    2.0
Name: 1, Length: 3993, dtype: float64


In [23]:
df_act.iloc[:, 1] = df_act.iloc[:, 1].astype(int) # [:-2]

print(df_act.iloc[:, 1])



0       1.0
1       3.0
2       2.0
3       3.0
4       2.0
       ... 
3988    3.0
3989    2.0
3990    2.0
3991    3.0
3992    2.0
Name: 1, Length: 3993, dtype: float64


In [24]:
print(df_act.iloc[:, 2])


0       2
1       5
2       1
3       1
4       1
       ..
3988    2
3989    1
3990    1
3991    3
3992    5
Name: 2, Length: 3993, dtype: object


In [27]:
def try_convert_int(x):
    try:
        f = float(x)
        i = int(f)
        return i if f == i else x  # 整数だったらintにする、そうでなければそのまま
    except:
        return x  
    
df_act.iloc[:, 1] = df_act.iloc[:, 1].map(try_convert_int)

print(df_act.iloc[:, 1])


0       1.0
1       3.0
2       2.0
3       3.0
4       2.0
       ... 
3988    3.0
3989    2.0
3990    2.0
3991    3.0
3992    2.0
Name: 1, Length: 3993, dtype: float64


In [31]:
df_act.iloc[:, 1] = df_act.iloc[:, 1].astype(str).str.split('.').str[0]
print(df_act.iloc[:, 1])

0       1
1       3
2       2
3       3
4       2
       ..
3988    3
3989    2
3990    2
3991    3
3992    2
Name: 1, Length: 3993, dtype: object


/var/folders/_k/qgnv6b3961q5nz5l1v78bw080000gn/T/ipykernel_38253/1952329140.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       1
1       3
2       2
3       3
4       2
       ..
3988    3
3989    2
3990    2
3991    3
3992    2
Name: 1, Length: 3993, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_act.iloc[:, 1] = df_act.iloc[:, 1].astype(str).str.split('.').str[0]


In [45]:
df_act = pd.read_csv(os.path.join(input_path, 'act_weekly_filled_structured.csv')) #, header=None)
df_time = pd.read_csv(os.path.join(input_path, 'time_weekly_filled_structured.csv'))#, header=None)
df_act.iloc[:, 1] = df_act.iloc[:, 1].astype(str).str.split('.').str[0] # 最初の行動を整数に

# print(df_act.iloc[:, 1])
print(df_act)

          0  1  2     3     4     5     6     7     8     9  ...   78   79  \
0     23002  3  5     2     3     2     1  <s1>     3     2  ...  <p>  <p>   
1     23002  2  1     3     2     2     2     1  <s1>     3  ...  <p>  <p>   
2     23003  3  1  <s1>   955  <s2>   955  <s3>     3     1  ...  <p>  <p>   
3     23003  2  1  <s1>   955  <s2>   955  <s3>   955  <s4>  ...  <p>  <p>   
4     23003  2  1  <s1>   955  <s2>   955  <s3>   955  <s4>  ...  <p>  <p>   
...     ... .. ..   ...   ...   ...   ...   ...   ...   ...  ...  ...  ...   
3987  21402  3  2     1  <s1>   955  <s2>   955  <s3>   955  ...  <p>  <p>   
3988  21402  2  1  <s1>   955  <s2>   955  <s3>   955  <s4>  ...  <p>  <p>   
3989  21415  2  1  <s1>   955  <s2>   955  <s3>   955  <s4>  ...  <p>  <p>   
3990  21457  3  3     5     1  <s1>   955  <s2>   955  <s3>  ...  <p>  <p>   
3991  21550  2  5     2     1  <s1>   955  <s2>   955  <s3>  ...  <p>  <p>   

       80   81   82   83   84   85   86   87  
0     <p>  <p>  

/var/folders/_k/qgnv6b3961q5nz5l1v78bw080000gn/T/ipykernel_38253/3325456784.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       3
1       2
2       3
3       2
4       2
       ..
3987    3
3988    2
3989    2
3990    3
3991    2
Name: 1, Length: 3992, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_act.iloc[:, 1] = df_act.iloc[:, 1].astype(str).str.split('.').str[0] # 最初の行動を整数に


In [46]:
df_act_val = df_act.iloc[1:, 1:].values.astype(str)
nunique_act = np.unique(df_act_val)
# actn = len(np.unique(df_act.iloc[:, 1:].values))
print('act num:', len(nunique_act)) 
print(nunique_act)

act num: 19
['1' '1.0' '2' '2.0' '3' '3.0' '4' '4.0' '5' '5.0' '955' '<p>' '<s1>'
 '<s2>' '<s3>' '<s4>' '<s5>' '<s6>' '<s>']


In [49]:
print(df_act)
def to_int_if_possible(x):
    try:
        f = float(x)
        i = int(f)
        return i if f == i else x  # 2.0 → 2, 2.5 はそのまま（必要なら切り捨てにしても可）
    except:
        return x  # 変換できない文字列（例：'<s1>'など）はそのまま返す

df_act.iloc[:, 1:] = df_act.iloc[:, 1:].applymap(to_int_if_possible)

          0  1  2     3     4     5     6     7     8     9  ...   78   79  \
0     23002  3  5     2     3     2     1  <s1>     3     2  ...  <p>  <p>   
1     23002  2  1     3     2     2     2     1  <s1>     3  ...  <p>  <p>   
2     23003  3  1  <s1>   955  <s2>   955  <s3>     3     1  ...  <p>  <p>   
3     23003  2  1  <s1>   955  <s2>   955  <s3>   955  <s4>  ...  <p>  <p>   
4     23003  2  1  <s1>   955  <s2>   955  <s3>   955  <s4>  ...  <p>  <p>   
...     ... .. ..   ...   ...   ...   ...   ...   ...   ...  ...  ...  ...   
3987  21402  3  2     1  <s1>   955  <s2>   955  <s3>   955  ...  <p>  <p>   
3988  21402  2  1  <s1>   955  <s2>   955  <s3>   955  <s4>  ...  <p>  <p>   
3989  21415  2  1  <s1>   955  <s2>   955  <s3>   955  <s4>  ...  <p>  <p>   
3990  21457  3  3     5     1  <s1>   955  <s2>   955  <s3>  ...  <p>  <p>   
3991  21550  2  5     2     1  <s1>   955  <s2>   955  <s3>  ...  <p>  <p>   

       80   81   82   83   84   85   86   87  
0     <p>  <p>  

/var/folders/_k/qgnv6b3961q5nz5l1v78bw080000gn/T/ipykernel_38253/1871717435.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_act.iloc[:, 1:] = df_act.iloc[:, 1:].applymap(to_int_if_possible)


In [50]:
df_act_val = df_act.iloc[1:, 1:].values.astype(str)
nunique_act = np.unique(df_act_val)
# actn = len(np.unique(df_act.iloc[:, 1:].values))
print('act num:', len(nunique_act)) 
print(nunique_act)

act num: 14
['1' '2' '3' '4' '5' '955' '<p>' '<s1>' '<s2>' '<s3>' '<s4>' '<s5>' '<s6>'
 '<s>']


In [39]:

df_time_val = df_time.iloc[1:, 1:].values.astype(str)
nunique_time = np.unique(df_time_val)
# actn = len(np.unique(df_act.iloc[:, 1:].values))
print('time num:', len(nunique_time)) 
print(nunique_time)

time num: 33
['0' '1' '10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '2' '20' '21'
 '22' '23' '25' '3' '4' '5' '6' '7' '8' '9' '<p>' '<s1>' '<s2>' '<s3>'
 '<s4>' '<s5>' '<s6>' '<s>']
